# Challenge 7 - Fight Fire with Data
## Random Forest Model to Predict Fire Spread

The user will be using a Jupyter Notebook to run code that was developed in R. First, the user will check to see if the wind speed and brightness are correlated with the speed that the fire spreads derived from the satellite data. The input data has been prepared for you. Next, the user will run the code that creates a model (random forest) using the features they select (windspeed and brightness) as the inputs and estimates the speed of spread as the target variable (speed of spread). They will train a model, record the Root Mean Squared Error, and save the model into a deployable format also known as Predictive Model Markup Language (PMML). 


## Install and Load Packages

In [ ]:
# Install and load packages
library(projectLib)
project <- projectLib::Project$new(,"<ProjectId>", "<ProjectToken>")

library(randomForest)
library(caret)
library(data.table)
library(devtools)

install_git("git://github.com/jpmml/r2pmml.git")
library(r2pmml)

## Get and View Data

In [ ]:
# Get data
fireData = fread('https://raw.githubusercontent.com/watsonwarriors2020/challenges/master/Challenge_7_Merged_Data.csv'
                 , data.table=FALSE
                 , header = TRUE
                 , stringsAsFactors = FALSE)

# Attach the column names
attach(fireData)

# Show data structure
str(fireData)

## Train a Random Forest Model and Display Accuracy

In [11]:
# Train model
set.seed(556)
modFit_rf <- randomForest(speed_mph~brightness + frp + WindSpeedMph + SurfaceTemperatureFahrenheit,
                          data = fireData,
                          # nodesize = 1,
                          ntree = 20,
                          trControl = trainControl(method = "cv", number = 10)
                          )

print(modFit_rf)

#RMSE
round(postResample(fireData$speed_mph, predict(modFit_rf, fireData)),4)


Call:
 randomForest(formula = speed_mph ~ brightness + frp + WindSpeedMph +      SurfaceTemperatureFahrenheit, data = fireData, ntree = 20,      trControl = trainControl(method = "cv", number = 10)) 
               Type of random forest: regression
                     Number of trees: 20
No. of variables tried at each split: 1

          Mean of squared residuals: 0.09213779
                    % Var explained: 23.9


RMSE Rsquared      MAE 
  0.1843   0.8475   0.0179

## Export Predictive Model Markup Language file

In [ ]:
# Export the model to PMML
r2pmml(modFit_rf, "modFit_rf.pmml")
#Export the PMML file to project storage
project$save_data('modFit_rf.xml', "modFit_rf.pmml", overwrite=TRUE)

## Complete Challenge

In [ ]:
# Verification steps here
